In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import sys
import pandas as pd




In [401]:
dataPred = pd.read_csv("../Prediction_Files/predictions_SVR_RBF.csv")

# reading predictions and transforming data

In [402]:
dataPred.Hours = ((dataPred.Hours)*23)+0
dataPred.Minutes = ((dataPred.Minutes)*59)+0
dataPred.Seconds = ((dataPred.Seconds)*59.999805)+0.102506

In [403]:
def rowToTime(row):
    timeAtMoment = row.Hours*3600*1000
    timeAtMoment += row.Minutes*60*1000
    timeAtMoment += row.Seconds*1000
    return timeAtMoment

In [404]:
resetAddTime = 23*3600*1000
resetAddTime += 59*60*1000
resetAddTime += 60*1000

In [405]:
dataPred

,Hours,Minutes,Seconds,Latency,Cost,Reliability,Friday,Monday,Saturday,Sunday,...,Server_7.0,Server_8.0,Tactic_1.0,Tactic_2.0,Tactic_3.0,Tactic_4.0,Tactic_5.0,predicted_Latency,predicted_Cost,predicted_Reliability
0,16.0,26.0,4.933805,0.005321,0.274131,1,0,0,0,1,...,0,0,1,0,0,0,0,0.047566,0.333513,1.0
1,16.0,26.0,6.696362,0.001634,0.321750,1,0,0,0,1,...,0,0,0,1,0,0,0,0.047905,0.333649,1.0
2,16.0,26.0,6.721756,0.000023,0.256113,1,0,0,0,1,...,0,0,0,0,1,0,0,0.073536,0.240651,1.0
3,16.0,26.0,43.598072,0.034535,0.323037,1,0,0,0,1,...,0,0,0,0,0,1,0,0.045586,0.234887,1.0
4,16.0,26.0,43.849946,0.000235,0.324324,1,0,0,0,1,...,0,0,0,0,0,0,1,0.209354,0.095800,1.0
5,16.0,27.0,26.762578,0.040285,0.037323,1,0,0,0,1,...,0,0,1,0,0,0,0,0.052832,0.327733,1.0
6,16.0,27.0,28.847418,0.001932,0.330759,1,0,0,0,1,...,0,0,0,1,0,0,0,0.053350,0.362148,1.0
7,16.0,27.0,28.883291,0.000033,0.342342,1,0,0,0,1,...,0,0,0,0,1,0,0,0.067885,0.269579,1.0
8,16.0,28.0,5.867204,0.034732,0.329472,1,0,0,0,1,...,0,0,0,0,0,1,0,0.035780,0.225097,1.0
9,16.0,28.0,6.122377,0.000238,0.326898,1,0,0,0,1,...,0,0,0,0,0,0,1,0.064406,0.155999,1.0


### this is converting hours, minutes, seconds to a millisecond time series

In [406]:
cumulativeTime = [rowToTime(dataPred.iloc[0])]

for i in range(1,len(dataPred)):
    timeAtMoment = rowToTime(dataPred.iloc[i])
    lastTimeMoment = rowToTime(dataPred.iloc[i-1])
    if timeAtMoment > lastTimeMoment:
        cumuTime = cumulativeTime[i-1] + (timeAtMoment - lastTimeMoment)
        cumulativeTime.append(cumuTime)
    else:
        cumulativeTime.append(cumulativeTime[i-1]+(timeAtMoment+resetAddTime)-lastTimeMoment)
#         print(cumulativeTime[i])

In [407]:
cumulativeTime = cumulativeTime- cumulativeTime[0]
cumulativeTime

array([0.00000000e+00, 1.76255727e+03, 1.78795063e+03, ...,
       1.85630294e+10, 1.85630679e+10, 1.85630688e+10])

In [408]:
cumulativeTime[14]-cumulativeTime[13]

252.9571378827095

### shifting and transforming data to a range of 0-50

In [409]:
def shiftTransformCol50(data):
    return (data + abs(data.min()))*50

In [410]:
newData = np.stack((cumulativeTime, shiftTransformCol50(dataPred.Latency), shiftTransformCol50(dataPred.Cost), dataPred.Reliability, shiftTransformCol50(dataPred.predicted_Latency), shiftTransformCol50(dataPred.predicted_Cost), round(dataPred.predicted_Reliability)),axis = 1)
newData = pd.DataFrame(newData, columns=['Timestamp', 'Latency', 'Cost', 'Reliability', 'predLatency', 'predCost', 'predReliability' ])


In [411]:
newData.describe()

,Timestamp,Latency,Cost,Reliability,predLatency,predCost,predReliability
count,1.148900e+04,11489.000000,11489.000000,11489.000000,11488.000000,11488.000000,11488.0
mean,9.310318e+09,1.003451,13.713010,0.996170,4.577942,16.796407,1.0
std,5.480846e+09,2.463576,5.591597,0.061769,1.185646,3.412141,0.0
min,0.000000e+00,0.000000,0.514800,0.000000,0.000000,7.640635,1.0
25%,4.380477e+09,0.013071,14.028314,1.000000,3.899180,15.267381,1.0
50%,9.539345e+09,0.256169,16.344916,1.000000,4.396833,16.814550,1.0
75%,1.401013e+10,1.762668,16.537967,1.000000,4.996433,19.958280,1.0
max,1.856307e+10,42.415283,35.971686,1.000000,14.513115,23.200582,1.0


In [412]:
data = newData

In [413]:
def utilFunction(latency, cost, reward = 100, reliability = 1):
    epsilon = 0.00000000000000000000000000000000000000000000001

    if reliability == 0:
        return 0
    else:
        return ((reward)/(latency+cost+epsilon)) * reliability 
    


realUtilityMean = 4.138921e+06	
realUtilStd = 6.175693e+06	

In [414]:
lastUpdate = 0;
updateThreshold = 1000
utilitPredicted = []
utilityReal = []
award = []
lostUtility = []

In [415]:
# real time utility aware analysis tool

In [416]:
temp = data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp']
# type(temp.total_seconds())
# pd.to_datetime(data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp'], unit='ms')

In [417]:
for i in range(1,len(data)):
#     timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp']).total_seconds()
    timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp'])

    predictedUtil = utilFunction(data.iloc[i].predLatency, data.iloc[i].predCost,(float)(timeDeltaReward), data.iloc[i].predReliability )
    
    realUtility = utilFunction(data.iloc[i].Latency, data.iloc[i].Cost,(float)(timeDeltaReward), data.iloc[i].Reliability )
    utilitPredicted.append(predictedUtil)
    utilityReal.append(realUtility)
    if predictedUtil >= updateThreshold:
#         award.append(award[i-1] + realUtility-1);
        award.append(realUtility-updateThreshold);
        lostUtility.append(0)
    else:
#         award.append(award[i-1]);
        award.append(0);
    
        if realUtility >= updateThreshold:
            lostUtility.append(realUtility)
        else:
            lostUtility.append(0)


In [418]:
results = pd.DataFrame(np.stack([utilitPredicted,utilityReal,award,lostUtility],axis=1),columns=['predictedUtility','realUtility','award','lostUtility'])

In [419]:
results

,predictedUtility,realUtility,award,lostUtility
0,7.415394e+01,1.072988e+02,0.000000e+00,0.000000
1,1.244738e+00,1.943731e+00,0.000000e+00,0.000000
2,1.970427e+03,2.033319e+03,1.033319e+03,0.000000
3,1.262597e+01,1.527865e+01,0.000000e+00,0.000000
4,1.809172e+03,1.037093e+04,9.370930e+03,0.000000
5,8.186739e+01,1.234220e+02,0.000000e+00,0.000000
6,1.663543e+00,2.064515e+00,0.000000e+00,0.000000
7,2.085351e+03,2.002638e+03,1.002638e+03,0.000000
8,1.624122e+01,1.535870e+01,0.000000e+00,0.000000
9,1.350132e+03,7.833239e+03,6.833239e+03,0.000000


In [420]:
results.to_csv("rewardAwareUtility-SVR_RBF.csv")

In [305]:
cumulativeTime[20]-cumulativeTime[19]

13978.811928711832